In [3]:
import os
import itertools
import time
import json
import tqdm

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold, train_test_split

# Feature scaling, required for non-tree-based models
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats.mstats import winsorize

# Encoding categorical data for non-tree-based models
from sklearn.preprocessing import OneHotEncoder

In [4]:
goods_promote_price = pd.read_csv('../datasets/goods_promote_price.csv', low_memory=True, thousands=',')
goods_sku_relation = pd.read_csv('../datasets/goods_sku_relation.csv', low_memory=True, thousands=',')
goodsale = pd.read_csv('../datasets/goodsale.csv', low_memory=True, thousands=',')
goodsdaily = pd.read_csv('../datasets/goodsdaily.csv', low_memory=True, thousands=',')
marketing = pd.read_csv('../datasets/marketing.csv', low_memory=True, thousands=',')
goodsinfo = pd.read_csv('../datasets/goodsinfo.csv', low_memory=True, thousands=',')

submit_example = pd.read_csv('../datasets/submit_example.csv', low_memory = True, thousands = ',')

MemoryError: 

In [52]:
#Mean Forecast

#Only May
mean_temp = goodsale.merge(submit_example, on = 'sku_id')
mean_temp = mean_temp[(mean_temp['data_date'] > 20170500) & (mean_temp['data_date'] < 20170600)]
mean_csv = mean_temp.groupby(by = 'sku_id', as_index = False).sum()


mean_csv['week1'] = mean_csv['goods_num'] * 7 / 31
mean_csv['week2'] = mean_csv['goods_num'] * 7 / 31
mean_csv['week3'] = mean_csv['goods_num'] * 7 / 31
mean_csv['week4'] = mean_csv['goods_num'] * 7 / 31
mean_csv['week5'] = mean_csv['goods_num'] * 7 / 31

mean_csv = mean_csv.drop('data_date', axis = 1)
mean_csv = mean_csv.drop('goods_num', axis = 1)
mean_csv = mean_csv.drop('goods_price', axis = 1)
mean_csv = mean_csv.drop('orginal_shop_price', axis = 1)



In [53]:
a = pd.DataFrame({'sku_id': submit_example['sku_id']})

In [72]:
temp = pd.DataFrame({'sku_id': submit_example['sku_id']})


,data_date,goods_id,sku_id,goods_num,goods_price,orginal_shop_price
2,20170427,GrwmCH,SKDZNsaw,3,30.99,44.99
5,20170308,Gy8BSH,SKCDfdcA,1,13.99,23.99
11,20170417,G36lMO,SKQI5fKn,2,4.99,5.99
18,20170415,GcLF2n,SKD1kwuZ,1,6.99,14.99
20,20170430,GgFkum,SKU8fJ6V,1,13.99,22.99
21,20170328,GpITvO,SK7vluQg,2,6.99,6.99
23,20170317,GNwYlU,SKT4KivO,1,10.99,22.99
31,20170411,Gz0hOa,SKkBmi2q,3,11.99,16.99
51,20170323,Gw6uSW,SKro4f21,1,6.99,13.99
65,20170424,GzuZ4d,SKId9iul,2,15.99,26.99


In [63]:
mean_csv = pd.merge(temp, mean_csv, on = 'sku_id', how = 'outer')
mean_csv.fillna(0, inplace=True)
mean_csv.to_csv('../submissions/mean_submission.csv')

In [ ]:
#2018.10.17 For Test
mean_temp = goodsale.merge(submit_example, on = 'sku_id')
sum_temp = mean_temp[(mean_temp['data_date'] >= 20180124) & (mean_temp['data_date'] < 20180400)]
sum_temp.groupby(by = 'data_date', as_index = False).sum()
sum_temp

In [67]:
#Mean Forecast

#Nearest 44 Days
mean_temp = goodsale.merge(submit_example, on = 'sku_id')
mean_temp = mean_temp[(mean_temp['data_date'] > 20180200) & (mean_temp['data_date'] < 20180400)]
mean_csv = mean_temp.groupby(by = 'sku_id', as_index = False).sum()

day = 44

mean_csv['week1'] = mean_csv['goods_num'] * 7 / day
mean_csv['week2'] = mean_csv['goods_num'] * 7 / day
mean_csv['week3'] = mean_csv['goods_num'] * 7 / day
mean_csv['week4'] = mean_csv['goods_num'] * 7 / day
mean_csv['week5'] = mean_csv['goods_num'] * 7 / day

mean_csv = mean_csv.drop('data_date', axis = 1)
mean_csv = mean_csv.drop('goods_num', axis = 1)
mean_csv = mean_csv.drop('goods_price', axis = 1)
mean_csv = mean_csv.drop('orginal_shop_price', axis = 1)

temp = pd.DataFrame({'sku_id': submit_example['sku_id']})
mean_csv = pd.merge(temp, mean_csv, on = 'sku_id', how = 'outer')
mean_csv.fillna(0, inplace=True)

mean_csv

,sku_id,week1,week2,week3,week4,week5
0,SKDtK67I,1.272727,1.272727,1.272727,1.272727,1.272727
1,SKpLKkIS,0.477273,0.477273,0.477273,0.477273,0.477273
2,SK2vQMpX,9.863636,9.863636,9.863636,9.863636,9.863636
3,SKY0RuBE,1.750000,1.750000,1.750000,1.750000,1.750000
4,SKvr2o3y,0.477273,0.477273,0.477273,0.477273,0.477273
5,SK4VKk3b,0.318182,0.318182,0.318182,0.318182,0.318182
6,SKDbZjmX,1.431818,1.431818,1.431818,1.431818,1.431818
7,SK9wkGUJ,0.318182,0.318182,0.318182,0.318182,0.318182
8,SKjQriLy,0.477273,0.477273,0.477273,0.477273,0.477273
9,SKGy1eQD,1.590909,1.590909,1.590909,1.590909,1.590909


In [68]:
mean_csv.to_csv('../submissions/mean_submission.csv')

In [103]:
#原数据分析

#原价
#goodsale[(goodsale['data_date'] > 20170300) & (goodsale['data_date'] <= 201780301)]
#打折
#goodsale[(goodsale['data_date'] > 20170300) & (goodsale['data_date'] <= 201880301) & (goodsale['goods_price'] < goodsale['orginal_shop_price'])]
#从20170301开始第一批销售
#到20180316最后一批销售
#20170301卖出10731单 == 其中8603单为打折商品
#20180301卖出14236单 == 其中12733单为打折商品

#全年一共7325028单 ， 其中4873747单只买了1件
#全年一共6280251打折单 ， 其中4131458单只买了1件

#goodsale[(goodsale['data_date'] > 20170300) & (goodsale['data_date'] <= 201880301)  & (goodsale['goods_num'] >= 5)]


#全年只卖出630742类sku_id的商品 ， 其中221213类只卖出一件 ， 156475类 大于等于10件 ， 27623类大于等于100件 ， 1955类大于等于1000件 ， 28类大于等于10000件
temp = goodsale.groupby(by = 'sku_id', as_index = False).sum()


In [108]:
temp[(temp['goods_num'] >= 10000)]

,sku_id,data_date,goods_num,goods_price,orginal_shop_price
10026,SK0zgq8K,7201679763,19742,2219.17,4611.40
14975,SK1U29xA,7685768874,13561,2287.42,2663.19
38258,SK3lCey0,6899125698,10089,3282.63,4521.55
39500,SK3siKA0,7584896219,12194,2647.66,3756.24
49878,SK4taRSU,7685768874,13304,5113.65,9837.55
63424,SK6FYOmA,7181430503,11189,1454.70,1776.44
65495,SK6Rv7Fd,5709066996,13577,1494.40,1695.17
72544,SK79ErpA,4216387456,13190,1050.91,1251.91
80432,SK7u26WG,7504232794,12521,941.12,1112.28
95445,SK9OW4H1,7040207625,12343,2303.16,3486.51


In [69]:
import numpy as np
import pandas as pd
df1=pd.DataFrame({'a':[1,2,3,4],
                  'b':[11,22,33,44],
                  'c':[111,222,333,444]})
df2=pd.DataFrame({'a':[1,2,3,4,5],
                  'e':[np.nan,3,4,6,8]})
print(df1)
print(df2)
df1.merge(df2, on = 'a')

   a   b    c
0  1  11  111
1  2  22  222
2  3  33  333
3  4  44  444
   a    e
0  1  NaN
1  2  3.0
2  3  4.0
3  4  6.0
4  5  8.0


,a,b,c,e
0,1,11,111,NaN
1,2,22,222,3.0
2,3,33,333,4.0
3,4,44,444,6.0


In [36]:
#Mean Forecast

#Nearest 15-45 Days
mean_temp = goodsale.merge(submit_example, on = 'sku_id')
mean_temp = mean_temp[(mean_temp['data_date'] > 20180124) & (mean_temp['data_date'] < 20180400)]
mean_csv = mean_temp.groupby(by = 'sku_id', as_index = False).sum()

day = 51

mean_csv['week1'] = mean_csv['goods_num'] * 7 / day
mean_csv['week2'] = mean_csv['goods_num'] * 7 / day
mean_csv['week3'] = mean_csv['goods_num'] * 7 / day
mean_csv['week4'] = mean_csv['goods_num'] * 7 / day
mean_csv['week5'] = mean_csv['goods_num'] * 7 / day

mean_csv = mean_csv.drop('data_date', axis = 1)
mean_csv = mean_csv.drop('goods_num', axis = 1)
mean_csv = mean_csv.drop('goods_price', axis = 1)
mean_csv = mean_csv.drop('orginal_shop_price', axis = 1)

temp = pd.DataFrame({'sku_id': submit_example['sku_id']})
mean_csv = pd.merge(temp, mean_csv, on = 'sku_id', how = 'outer')
mean_csv.fillna(0, inplace=True)

mean_csv

,sku_id,week1,week2,week3,week4,week5
0,SKDtK67I,1.098039,1.098039,1.098039,1.098039,1.098039
1,SKpLKkIS,0.686275,0.686275,0.686275,0.686275,0.686275
2,SK2vQMpX,9.333333,9.333333,9.333333,9.333333,9.333333
3,SKY0RuBE,1.784314,1.784314,1.784314,1.784314,1.784314
4,SKvr2o3y,0.411765,0.411765,0.411765,0.411765,0.411765
5,SK4VKk3b,0.823529,0.823529,0.823529,0.823529,0.823529
6,SKDbZjmX,1.235294,1.235294,1.235294,1.235294,1.235294
7,SK9wkGUJ,0.411765,0.411765,0.411765,0.411765,0.411765
8,SKjQriLy,0.411765,0.411765,0.411765,0.411765,0.411765
9,SKGy1eQD,1.921569,1.921569,1.921569,1.921569,1.921569


In [37]:
mean_csv.to_csv('../submissions/mean_submission.csv')

In [27]:
mean_temp = goodsale.merge(submit_example, on = 'sku_id')
mean_temp = mean_temp[(mean_temp['data_date'] > 20180127) & (mean_temp['data_date'] < 20180400)]

mean_temp.drop_duplicates(['data_date']).shape

(48, 11)